In [1]:
import sqlite3
import pandas as pd
import plotly.express as px

### FETCH DATA 

In [2]:
conn = sqlite3.connect('legal_documents_ecommerce.db')    

# NOTE item revenue is for the optional questions
# NOTE email is omitted 
# NOTE 
query = """
SELECT
    C.customer_id,
    C.first_name,
    C.last_name,
    C.registration_date,
    O.order_id,
    O.order_date,
    O.total_amount,
    P.product_name,
    P.category,
    P.price,
    OI.quantity,
    OI.unit_price,
    OI.quantity * OI.unit_price AS item_revenue 
FROM   Customers     AS C
JOIN   Orders        AS O  ON O.customer_id  = C.customer_id
JOIN   Order_items   AS OI ON OI.order_id    = O.order_id
JOIN   Products      AS P  ON P.product_id   = OI.product_id;
"""

df = pd.read_sql_query(query, conn)
df['order_date'] = pd.to_datetime(df['order_date'])
df['registration_date'] = pd.to_datetime(df['registration_date'])
conn.close()
df

,customer_id,first_name,last_name,registration_date,order_id,order_date,total_amount,product_name,category,price,quantity,unit_price,item_revenue
0,89,Daniel,Watts,2024-07-17,1,2024-12-11,336.46,Copyright Assignment,Intellectual Property,148.06,2,148.06,296.12
1,89,Daniel,Watts,2024-07-17,1,2024-12-11,336.46,Property Management Agreement,Real Estate,40.34,1,40.34,40.34
2,120,Brooke,Henderson,2024-02-29,2,2025-03-11,843.91,Prenuptial Agreement,Personal,259.84,1,259.84,259.84
3,120,Brooke,Henderson,2024-02-29,2,2025-03-11,843.91,Property Management Agreement,Real Estate,40.34,2,40.34,80.68
4,120,Brooke,Henderson,2024-02-29,2,2025-03-11,843.91,Partnership Agreement,Business,90.14,1,90.14,90.14
...,...,...,...,...,...,...,...,...,...,...,...,...,...
900,120,Brooke,Henderson,2024-02-29,300,2024-08-20,2308.46,LLC Formation Documents,Business,141.89,1,141.89,141.89
901,120,Brooke,Henderson,2024-02-29,300,2024-08-20,2308.46,Lease Agreement,Real Estate,185.35,3,185.35,556.05
902,120,Brooke,Henderson,2024-02-29,300,2024-08-20,2308.46,Patent Application,Intellectual Property,175.23,3,175.23,525.69
903,120,Brooke,Henderson,2024-02-29,300,2024-08-20,2308.46,Eviction Notice,Real Estate,141.75,2,141.75,283.50


### OVERVIEW 

In [3]:
print(f"dataset shape: {df.shape}")
print(f"date range: {df['order_date'].min().date()} to {df['order_date'].max().date()}")
print("\noverview:")
print(df.head())

print("\nKPIs")
print(f"total customers: {df['customer_id'].nunique():,}")
print(f"total orders: {df['order_id'].nunique():,}")
print(f"total revenue: ${df['item_revenue'].sum():,.2f}")
print(f"average order value: ${df.groupby('order_id')['item_revenue'].sum().mean():.2f}")
print(f"orders per customer: {df['order_id'].nunique() / df['customer_id'].nunique():.2f}")
print(f"average items per order: {df.groupby('order_id').size().mean():.2f}")

dataset shape: (905, 13)
date range: 2024-01-08 to 2025-04-14

overview:
   customer_id first_name  last_name registration_date  order_id order_date  \
0           89     Daniel      Watts        2024-07-17         1 2024-12-11   
1           89     Daniel      Watts        2024-07-17         1 2024-12-11   
2          120     Brooke  Henderson        2024-02-29         2 2025-03-11   
3          120     Brooke  Henderson        2024-02-29         2 2025-03-11   
4          120     Brooke  Henderson        2024-02-29         2 2025-03-11   

   total_amount                   product_name               category   price  \
0        336.46           Copyright Assignment  Intellectual Property  148.06   
1        336.46  Property Management Agreement            Real Estate   40.34   
2        843.91           Prenuptial Agreement               Personal  259.84   
3        843.91  Property Management Agreement            Real Estate   40.34   
4        843.91          Partnership Agreement 

### ORDERS PER WEEK/MONTH ANALYSIS

In [4]:
# add week and month start columns for grouping
df['week_start'] = df['order_date'] - pd.to_timedelta(df['order_date'].dt.dayofweek, unit='d') # start of week 
df['month_start'] = df['order_date'].dt.to_period('M').dt.to_timestamp()   # start of month 

# week 
weekly_orders = (df.groupby('week_start')['order_id'].nunique().reset_index(name='orders'))
fig_week = px.line(weekly_orders, x='week_start', y='orders', markers=True, title='Weekly Order Volume')
fig_week.update_layout(xaxis_title='Week (Monday–Sunday)', yaxis_title='Number of Orders')
fig_week.show()

# monthly 
monthly_orders = (df.groupby('month_start')['order_id'].nunique().reset_index(name='orders'))
fig_month = px.bar(monthly_orders, x='month_start', y='orders', title='Monthly Order Volume')
fig_month.update_layout(xaxis_title='Month', yaxis_title='Number of Orders')
fig_month.show()


### PRODUCT ORDER FREQUENCY ANALYSIS

In [5]:
# count unique orders for each product
product_freq = (df.groupby('product_name')['order_id'].nunique().reset_index(name='orders').sort_values('orders', ascending=False))

top_n = 10                               
top_products_freq = product_freq.head(top_n)

fig_prod_freq = px.bar(top_products_freq, x='product_name', y='orders', 
                       title=f'Top {top_n} Products by Order Count', text='orders')

fig_prod_freq.update_layout(xaxis_title='Product', yaxis_title='Number of Orders', xaxis_tickangle=45)
fig_prod_freq.show()


### CUSTOMER VALUE ANALYSIS

In [6]:
customer_value = (df.groupby(['customer_id', 'first_name', 'last_name']).agg({
                     'item_revenue': 'sum', 
                     'order_id': 'nunique',
                     'quantity': 'sum'
                    }).reset_index().sort_values('item_revenue', ascending=False))

customer_value['customer_name'] = customer_value['first_name'] + ' ' + customer_value['last_name']
top_customers = customer_value.head(10)

fig1 = px.bar(top_customers, x='customer_name', y='item_revenue',title="Top 10 Customers by Total Revenue",
              labels={'item_revenue': 'Total Revenue ($)', 'customer_name': 'Customer'})
fig1.update_layout(xaxis_tickangle=45)
fig1.show()

### PRODUCT PERFORMANCE ANALYSIS  

In [7]:
category_performance = (df.groupby('category').agg({'item_revenue': 'sum','quantity': 'sum',
                           'order_id': 'nunique'
                       }).reset_index().sort_values('item_revenue', ascending=False))

fig2 = px.pie(category_performance, values='item_revenue', names='category',
              title="Revenue Distribution by Product Category")
fig2.show()


In [8]:
top_products = (df.groupby(['product_name', 'category']).agg({'item_revenue': 'sum', 'quantity': 'sum'})
               .reset_index().sort_values('item_revenue', ascending=False).head(10))

fig3 = px.bar(top_products, x='product_name', y='item_revenue', color='category',title="Top Products by Revenue")
fig3.update_layout(xaxis_tickangle=45)
fig3.show()